In [ ]:
from dept import *
from datetime import datetime, timedelta

# AWS: boto3

In [ ]:
import boto3

# read config file
aws_config = read_file('configs/aws.json')
aws_session = boto3.Session(
    aws_access_key_id=aws_config.get('aws_access_key_id'),
    aws_secret_access_key=aws_config.get('aws_secret_access_key'),
    region_name=aws_config.get('region_name')
    )

# bucket name used in the tutorial
bucket_name = aws_config.get('bucket_name')

- **CLIENT** - uses interaction with AWS services via functions 
- **RESOURCE** - uses interaction with AWS services via objects 
- **PAGINATORS** - provides a mean to list objects and attributes in chunks (pages)
- **WAITERS** - provides a way to regularly check for a specific condition to be met

## boto3: S3 client

In [ ]:
# instantiate S3 client
s3_client = aws_session.client('s3')

In [ ]:
# create a new bucket
s3_client.create_bucket(Bucket=bucket_name)

In [ ]:
# list S3 buckets
print_dict(s3_client.list_buckets())

In [ ]:
# create a test file
file_path = norm_path('sandbox/test_file.txt')
content = 'this is a test file.'

write_file(content=content, file_path=file_path)

In [ ]:
# upload file
file_key='test-upload-file'

s3_client.upload_file(
    Filename=file_path,
    Bucket=bucket_name,
    Key=file_key
    )

In [ ]:
# download file from S3
destination_file_path = norm_path('sandbox/test_file_download.txt')

s3_client.download_file(
    Filename=destination_file_path,
    Bucket=bucket_name,
    Key=file_key
    )

In [ ]:
# delete S3 object
s3_client.delete_object(
    Bucket=bucket_name,
    Key=file_key
    )

## boto3: S3 resource

In [ ]:
# instantiate S3 resource connector
s3 = aws_session.resource('s3')

In [ ]:
# create a bucket via S3 resource
s3.create_bucket(Bucket=bucket_name)

In [ ]:
# interact with S3 bucket 
bucket = s3.Bucket(name=bucket_name)

# upload file
bucket.upload_file(
    Filename=file_path,
    Key=file_key
)

In [ ]:
# list all objects
print(*bucket.objects.all())

In [ ]:
# download file from a bucket
bucket.download_file(
    Filename=destination_file_path,
    Key=file_key
)

In [ ]:
# filter bucket objects
print(*bucket.objects.filter(Prefix='test'))

## boto3: deleting S3 buckets

In [ ]:
# get S3 bucket 
bucket = s3.Bucket(name=bucket_name)

# delete all S3 bucket objects -> only an empty bucket can be deleted
bucket.objects.all().delete()

# delete empty bucket
bucket.delete()

## boto3: paginators

- provides a mean to list objects in chunks (pages)

In [ ]:
# get paginator -> S3 client is required!
paginator = s3_client.get_paginator('list_objects_v2')
results = paginator.paginate(
    Bucket=bucket_name
)

In [ ]:
# iterate over results
for id, item in enumerate(results.search('Contents')):
    print(f"index: {id}")
    print_dict(item)

## boto3: waiters

- regularly checks for a specific condition to be met

In [ ]:
# define waiter parameters
waiter = s3_client.get_waiter('bucket_exists')
wait_config = {
    'Delay': 10,             # check every 10 seconds 
    'MaxAttempts': 6         # exit after 6 unsuccessful attemps
}

# testing bucket name
test_bucket_name = 'very-weird-bucket-to-check'

In [ ]:
print(f'waiting for a bucket: {test_bucket_name}')
waiter.wait(
    Bucket=test_bucket_name,
    WaiterConfig=wait_config
    )

## boto3: file sharing

- using presigned URLs

In [ ]:
url = s3_client.generate_presigned_url(
    ClientMethod='get_object',
    Params={
        'Bucket': bucket_name, 
        'Key': file_key,
        },
    ExpiresIn=120                 # expires in 120 seconds
)

## boto3: cost monitoring

In [ ]:
# instantiate S3 client
ce_client = aws_session.client('ce')

In [ ]:
# define reporting period
start_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')
print(f'period: {start_date} // {end_date}')

# boto3 client format
reporting_period = {
        'Start': start_date,
        'End': end_date
    }

In [ ]:
# get cost and usage statistics
response = ce_client.get_cost_and_usage(
    TimePeriod=reporting_period,
    Granularity='MONTHLY',
    Metrics=['UnblendedCost', 'UsageQuantity']
)

# explore costs
for item in response['ResultsByTime']:
    print(f"period: {item['Timeperiod']}")
    print(f"total cost: {item['Total']['UnblendedCost']}")
    print("\n")

In [ ]:
# get list of used services
response = ce_client.get_dimension_values(
    TimePeriod=reporting_period,
    Dimension='SERVICE'
)

for service in response['DimensionValues']: 
    print(service['Value'])

In [ ]:
# get cost and usage statistics
response = ce_client.get_cost_and_usage(
    TimePeriod=reporting_period,
    Granularity='MONTHLY',
    Metrics=['UnblendedCost'],
    GroupBy=[
        {
            "Type":"DIMENSION",
            "Key":"SERVICE"
        }
    ]
)

# explore costs
for item in response['ResultsByTime']:
    print(f"period: {item['Timeperiod']}")

    for group in item['Groups']:
        service_name = group['Keys'][0]
        cost = group['Metrics']['UnblendedCost']['Amount']
        print(f"{service_name}: ${cost}")
    print("\n") 

## boto3: cost forecast

In [ ]:
# define forecast period
start_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=31)).strftime('%Y-%m-%d')
print(f'period: {start_date} // {end_date}')

# boto3 client format
forecast_period = {
        'Start': start_date,
        'End': end_date
    }

In [ ]:
# collect forecast
result = ce_client.get_cost_forecast(
    TimePeriod=forecast_period,
    Metric='UNBLENDED_COST',
    Granularity='MONTHLY'
)

## boto3: EC2

- **IMPORTANT**: to deploy an EC2 instance, server image ID is required \
  -> use AWS console to get the available IDs \
  `AWS console -> EC2 -> Launch an instance`

In [ ]:
ec2 = aws_session.resource('ec2')

In [ ]:
# deploy instance
instance = ec2.create_instances(
    MinCount=1,
    MaxCount=1,
    InstanceType="t3.micro",
    ImageId="ami-0014ce3e52359afbd"
)
# list all instances
print(*ec2.instances.all())

In [ ]:
instances = [i.id for i in instance]

In [ ]:
# check instance attributes
ec2_client = aws_session.client('ec2')
ec2_client.describe_instances(
    InstanceIds=instances
    )

In [ ]:
# stop an instance (not terminating it)
ec2_client.stop_instances(
    InstanceIds=instances
)

In [ ]:
# restart stopped instances
ec2_client.start_instances(
    InstanceIds=instances
)

In [ ]:
# terminate instances
ec2_client.terminate_instances(
    InstanceIds=instances
)

### EC2: SSH connection

- **paramiko** library is used to establish an SSH connection to EC2 instance

In [ ]:
# esteblish connections
ec2_client = aws_session.client('ec2')
ec2 = aws_session.resource('ec2')

# define key pair identifier and its storage location
key_pair_id = 'course_key_pair'
key_pair_path = f'./sandbox/{key_pair_id}.pem'

In [ ]:
# create SSH key pair
key_pair = ec2_client.create_key_pair(
    KeyName=key_pair_id
)

In [ ]:
# create a file containing the key pair
write_file(
    content=key_pair['KeyMaterial'], 
    file_path=key_pair_path
    )

In [ ]:
# list security groups
security_groups = ec2_client.describe_security_groups()
print_dict(*security_groups['SecurityGroups'])

In [ ]:
# deploy instance with security group and SSH connection
instance = ec2.create_instances(
    MinCount=1,
    MaxCount=1,
    InstanceType="t3.micro",
    ImageId="ami-0014ce3e52359afbd",
    KeyName=key_pair_id,
    SecurityGroupIds=['sg-07662f87cb90b2d23']
)

In [ ]:
# get instance IDs
instances = [i.id for i in instance]
instances

In [ ]:
# get instance descriptions
instance_descriptions = ec2_client.describe_instances(
    InstanceIds=instances
)
instance_descriptions

In [ ]:
# extract IP addresses
ip_mapping = {}
for inst in instance_descriptions['Reservations'][0]['Instances']:
    ip_mapping[inst['InstanceId']] = inst['PrivateIpAddress']

ip_mapping

In [ ]:
# !pip install paramiko
import paramiko

In [ ]:
# establish ssh client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_key = paramiko.RSAKey.from_private_key_file(
    filename= key_pair_path
    )

In [ ]:
ssh_key

In [ ]:
# get IP address of the first instance
ip = ip_mapping.get(instances[0])
ip

In [ ]:
# connect to EC2 instance
ssh.connect(
    hostname=ip,
    username='ubuntu',
    pkey=ssh_key
    )

In [ ]:
# execute shell command and read output
stdin, stdout, stderr = ssh.exec_command('ls')
stdout.read()

In [ ]:
# terminate instances
ec2_client.terminate_instances(
    InstanceIds=instances
)